In [1]:
# This takes SDSS FITS spectra and separates out the single-epoch spectra and writes them out again as 
# individual ascii files

# Created 2022 Dec. 1 by E.S.

In [1]:
from astropy.io import fits
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import numpy as np
import glob
import os

%matplotlib qt

In [2]:
stem = "/Users/bandari/Documents/git.repos/rrlfe/src/sdss_original_single_epoch_fits/"
stem_notebooks = "/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/"

In [3]:
file_list = glob.glob(stem + "*fits")

In [4]:
'''
# how many spectra total?
total_n = 0
for file_name in file_list:
    print()
    hdul = fits.open(file_name)
    
    total_n += len(hdul)-4
'''

'\n# how many spectra total?\ntotal_n = 0\nfor file_name in file_list:\n    print()\n    hdul = fits.open(file_name)\n    \n    total_n += len(hdul)-4\n'

In [5]:
'''
Name      Type      Comment
flux      float32   coadded calibrated flux [10-17 ergs/s/cm2/Å]
loglam    float32   log10(wavelength [Å])
ivar      float32   inverse variance of flux
and_mask  int32     AND mask
or_mask   int32     OR mask
wdisp     float32   wavelength dispersion in pixel=dloglam units
sky       float32   subtracted sky flux [10-17 ergs/s/cm2/Å]
model     float32   pipeline best model fit used for classification and redshift
'''

'\nName      Type      Comment\nflux      float32   coadded calibrated flux [10-17 ergs/s/cm2/Å]\nloglam    float32   log10(wavelength [Å])\nivar      float32   inverse variance of flux\nand_mask  int32     AND mask\nor_mask   int32     OR mask\nwdisp     float32   wavelength dispersion in pixel=dloglam units\nsky       float32   subtracted sky flux [10-17 ergs/s/cm2/Å]\nmodel     float32   pipeline best model fit used for classification and redshift\n'

In [6]:
# make new abcissae for interpolating the plots
'''
blue_0 = 3850.
blue_1 = 5970.
num_steps_blue = int(np.floor((blue_1-blue_0)/1.4))
blue_abcissa_interp = np.add(blue_0,1.4*np.arange(0,num_steps_blue+1,1))

red_0 = 6030.
red_1 = 9000.
num_steps_red = int(np.floor((red_1-red_0)/1.4))
red_abcissa_interp = np.add(red_0,1.4*np.arange(0,num_steps_red+2,1))

#test = np.linspace(blue_0,blue_1,num=num_steps,endpoint=False)
'''

'\nblue_0 = 3850.\nblue_1 = 5970.\nnum_steps_blue = int(np.floor((blue_1-blue_0)/1.4))\nblue_abcissa_interp = np.add(blue_0,1.4*np.arange(0,num_steps_blue+1,1))\n\nred_0 = 6030.\nred_1 = 9000.\nnum_steps_red = int(np.floor((red_1-red_0)/1.4))\nred_abcissa_interp = np.add(red_0,1.4*np.arange(0,num_steps_red+2,1))\n\n#test = np.linspace(blue_0,blue_1,num=num_steps,endpoint=False)\n'

In [7]:
# check that ascii files are identical with these

# blue end: interpolate between [3850,5970] at intervals of 1.4 angstroms such that 3900.00, 3901.40 [...] 4999.00
# red end: interpolate between [6030:]

#num_spec_array = np.zeros(len(file_list))

#stem1 = "/Users/bandari/Documents/git.repos/rrlfe/src/sdss_cosmic_rays_removed/"
#df = pd.read_csv(stem1 + "spec-1681-53172-0551g001.dat", names=["wavel", "flux", "error"], delim_whitespace=True)

for i in range(0,1):#len(file_list)):
    
    print(file_list[i])
    print(i)

    hdul = fits.open(file_list[i])
    
    num_spec_total = len(hdul)-4
    
    #num_spec_array[i] = num_spec_total
    
    #print(num_spec_total)
    
    # loop over blue-red pairs
    for num_blue in range(int(0.5*num_spec_total)):
        
        print("-----")
        
        # blue end
        idx_blue = 4+num_blue
        data_blue = hdul[idx_blue].data
        flux_blue = np.flip(data_blue["flux"]) # flip to make wavelength increasing
        wavel_blue = np.flip(np.power(10.,data_blue["loglam"]))
        sigma_blue = np.flip(np.divide(1.,np.sqrt(data_blue["ivar"])))
        #print("idx_blue",idx_blue)
        #print(np.median(np.divide(flux_blue,sigma_blue)))
                
        # red end
        idx_red = idx_blue+int(0.5*num_spec_total)
        data_red = hdul[idx_red].data
        flux_red = data_red["flux"]
        wavel_red = np.power(10.,data_red["loglam"])
        sigma_red = np.divide(1.,np.sqrt(data_red["ivar"]))
        #print("idx_red",idx_red)
        #print(np.median(np.divide(flux_red,sigma_red)))
        
        # make new abcissae for interpolating the plots
        blue_0 = 3850. # bluest of blue end
        red_1 = 9000. # reddest of red end
        num_steps = int(np.floor((red_1-blue_0)/1.4))
        abcissa_interp = np.add(blue_0-0.4,1.4*np.arange(0,num_steps+2,1)) # -0.4 to get the phase right
        
        # interpolate the spectra

        # blue region I ultimately want: [3850:5970]
        # red region I ultimately want: [6030:9000]
        # overlap region that needs to be averaged: [5970:6030]

        # indices of data points corresponding to abcissa [3850:6030] (note includes overlap region)
        idx_blue = np.where( np.logical_and(abcissa_interp > 3849., abcissa_interp < 6031.) )
        # indices of data points corresponding to abcissa [5970:9000] (note includes overlap region)
        idx_red = np.where( np.logical_and(abcissa_interp > 5969., abcissa_interp < 9001.) )
        # overlap region
        #idx_overlap = 

        flux_blue_interp = np.interp(x = abcissa_interp[idx_blue], xp = wavel_blue, fp = flux_blue)
        error_flux_blue_interp = np.interp(x = abcissa_interp[idx_blue], xp = wavel_blue, fp = sigma_blue)
        flux_red_interp = np.interp(x = abcissa_interp[idx_red], xp = wavel_red, fp = flux_red)
        error_flux_red_interp = np.interp(x = abcissa_interp[idx_red], xp = wavel_red, fp = sigma_red)
        
        # write out
        # cols:
        # [0] wavel_interp
        # [1] flux_blue_interp
        # [2] error_flux_blue_interp
        # [3] wavel_red_interp
        # [4] flux_red_interp
        # [5] error_flux_red_interp
        
        df_blue_write = pd.DataFrame(abcissa_interp[idx_blue], columns=["wavel_blue"])
        df_blue_write["flux_blue"] = flux_blue_interp
        df_blue_write["error_flux_blue"] = error_flux_blue_interp
        
        df_red_write = pd.DataFrame(abcissa_interp[idx_red], columns=["wavel_red"])
        df_red_write["flux_red"] = flux_red_interp
        df_red_write["error_flux_red"] = error_flux_red_interp
        
        # write csvs
        file_name_blue_this = stem_notebooks + os.path.basename(file_list[i]).split(".")[0] + '_g{:0>3}'.format(num_blue) + "_color_blue.csv"
        df_blue_write.to_csv(file_name_blue_this, index=False)
        print("Wrote",file_name_blue_this)
        
        file_name_red_this = stem_notebooks + os.path.basename(file_list[i]).split(".")[0] + '_g{:0>3}'.format(num_blue) + "_color_red.csv"
        df_red_write.to_csv(file_name_red_this, index=False)
        print("Wrote",file_name_red_this)
        
        # save
        '''
        plt.clf()
        fig = plt.figure(figsize=(15,5))
        plt.plot(wavel_blue,flux_blue)
        plt.plot(wavel_blue,sigma_blue)
        plt.plot(wavel_red,flux_red)
        plt.plot(wavel_red,sigma_red)
        plt.plot(df["wavel"],df["flux"])
        plt.axvline(x=5970, linestyle="--")
        plt.axvline(x=6030, linestyle="--")
        plt.ylim([0,100])
        plt.xlabel("wavel (angstrom)")
        plt.ylabel("flux")
        plt.xlim([4000,4150])
        #plt.savefig("./junk_data/" + os.path.basename(file_list[i]) + ".png")
        plt.show()
        '''

/Users/bandari/Documents/git.repos/rrlfe/src/sdss_original_single_epoch_fits/spec-2112-53534-0521.fits
0
-----
Wrote /Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/spec-2112-53534-0521_g000_color_blue.csv
Wrote /Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/spec-2112-53534-0521_g000_color_red.csv
-----
Wrote /Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/spec-2112-53534-0521_g001_color_blue.csv
Wrote /Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/spec-2112-53534-0521_g001_color_red.csv
-----
Wrote /Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/spec-2112-53534-0521_g002_color_blue.csv
Wrote /Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/spec-2112-53534-0521_g002_color_red.csv


/Users/bandari/anaconda3/envs/rrl_env/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in true_divide


In [13]:
# check an individual spectrum

i=400

print(file_list[i])
print(i)

hdul = fits.open(file_list[i])

print(hdul.info())

num_spec_total = len(hdul)-4

#num_spec_array[i] = num_spec_total
plt.clf()
print("num_spec_total",num_spec_total)

# loop over blue-red pairs
for num_blue in range(0,int(0.5*num_spec_total)):

    print("-----")

    # blue end
    idx_blue = 4+num_blue
    data_blue = hdul[idx_blue].data
    flux_blue = np.flip(data_blue["flux"]) # flip to make wavelength increasing
    wavel_blue = np.flip(np.power(10.,data_blue["loglam"]))
    sigma_blue = np.flip(np.divide(1.,np.sqrt(data_blue["ivar"])))
    #print("idx_blue",idx_blue)
    #print(np.median(np.divide(flux_blue,sigma_blue)))

    # red end
    idx_red = idx_blue+int(0.5*num_spec_total)
    data_red = hdul[idx_red].data
    flux_red = data_red["flux"]
    wavel_red = np.power(10.,data_red["loglam"])
    sigma_red = np.divide(1.,np.sqrt(data_red["ivar"]))
    #print("idx_red",idx_red)
    #print(np.median(np.divide(flux_red,sigma_red)))

    # make new abcissae for interpolating the plots
    blue_0 = 3850. # bluest of blue end
    red_1 = 9000. # reddest of red end
    num_steps = int(np.floor((red_1-blue_0)/1.4))
    abcissa_interp = np.add(blue_0-0.4,1.4*np.arange(0,num_steps+2,1)) # -0.4 to get the phase right

    # interpolate the spectra

    # blue region I ultimately want: [3850:5970]
    # red region I ultimately want: [6030:9000]
    # overlap region that needs to be averaged: [5970:6030]

    # indices of data points corresponding to abcissa [3850:6030] (note includes overlap region)
    idx_blue = np.where( np.logical_and(abcissa_interp > 3849., abcissa_interp < 6031.) )
    # indices of data points corresponding to abcissa [5970:9000] (note includes overlap region)
    idx_red = np.where( np.logical_and(abcissa_interp > 5969., abcissa_interp < 9001.) )
    # overlap region
    #idx_overlap = 

    flux_blue_interp = np.interp(x = abcissa_interp[idx_blue], xp = wavel_blue, fp = flux_blue)
    error_flux_blue_interp = np.interp(x = abcissa_interp[idx_blue], xp = wavel_blue, fp = sigma_blue)
    flux_red_interp = np.interp(x = abcissa_interp[idx_red], xp = wavel_red, fp = flux_red)
    error_flux_red_interp = np.interp(x = abcissa_interp[idx_red], xp = wavel_red, fp = sigma_red)
    
    plt.plot(wavel_blue,np.add(num_blue*100,flux_blue),label="read in")
    plt.plot(wavel_red,np.add(num_blue*100,flux_red),label="read in")
    plt.plot(abcissa_interp[idx_blue],np.add(num_blue*100,flux_blue_interp),label="interp")
    plt.plot(abcissa_interp[idx_red],np.add(num_blue*100,flux_red_interp),label="interp")

plt.legend()
plt.show()

/Users/bandari/Documents/git.repos/rrlfe/src/sdss_original_single_epoch_fits/spec-0730-52466-0488.fits
400
Filename: /Users/bandari/Documents/git.repos/rrlfe/src/sdss_original_single_epoch_fits/spec-0730-52466-0488.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     141   ()      
  1  COADD         1 BinTableHDU     26   3848R x 8C   [E, E, E, J, J, E, E, E]   
  2  SPECOBJ       1 BinTableHDU    262   1R x 126C   [6A, 4A, 16A, 23A, 16A, 8A, E, E, E, J, E, E, J, B, B, B, B, B, B, J, 22A, 19A, 19A, 22A, 19A, I, 3A, 3A, 1A, J, D, D, D, E, E, 19A, 8A, J, J, J, J, K, K, J, J, J, J, J, J, K, K, K, K, I, J, J, J, J, 5J, D, D, 6A, 21A, E, E, E, J, E, 24A, 10J, J, 10E, E, E, E, E, E, E, J, E, E, E, J, E, 5E, E, 10E, 10E, 10E, 5E, 5E, 5E, 5E, 5E, J, J, E, E, E, E, E, E, 25A, 21A, 10A, E, E, E, E, E, E, E, E, J, E, E, J, 1A, 1A, E, E, J, J, 1A, 5E, 5E]   
  3  SPZLINE       1 BinTableHDU     48   29R x 19C   [J, J, J, 13A, D, E, E, E, E, E, E, 

/Users/bandari/anaconda3/envs/rrl_env/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in true_divide
/Users/bandari/anaconda3/envs/rrl_env/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in true_divide
/Users/bandari/anaconda3/envs/rrl_env/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in true_divide
/Users/bandari/anaconda3/envs/rrl_env/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in true_divide


In [9]:
plt.plot(wavel_blue,flux_blue,label="blue")plt.show()